# Homework 1

### Installing packages

In [9]:
!pip install langchain_google_genai


## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.




In [10]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')

## Downloading receipts.zip
The codes below download and unzip receipts.zip from Google Drive. receipts.zip contains all images from the Fusion folder on BlackBoard.


In [11]:
import gdown
file_id = "1oe2FZd3ZTO7nrDqjCafNvxicl08oF8JF"
download_url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(download_url, "receipts.zip", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1oe2FZd3ZTO7nrDqjCafNvxicl08oF8JF
To: /content/receipts.zip
100%|██████████| 1.61M/1.61M [00:00<00:00, 37.4MB/s]


'receipts.zip'

In [ ]:
!unzip receipts.zip

Archive:  receipts.zip
replace receipt1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## 1. Helper functions

We need two functions
* image_to_base64 convert your jpg image into Base64 encoded string (basically a sequence of 64 characters to make your image easily transfered via API)
* get_image_data_url takes your jpg image, converting them into base64 string and construct a suitable input for GEMINI api call.

In [ ]:
import base64
import mimetypes

# Helper function to read and encode image
def image_to_base64(img_path):
    with open(img_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

# Helper function to encode local file to Base64 Data URL
def get_image_data_url(image_path):
    # Guess the mime type (e.g., image/png, image/jpeg) based on file extension
    mime_type, _ = mimetypes.guess_type(image_path)
    if mime_type is None:
        mime_type = "image/png" # Default fallback

    encoded_string = image_to_base64(image_path)

    # Construct the Data URL
    return f"data:{mime_type};base64,{encoded_string}"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=userdata.get('VERTEX_API_KEY'), # Ensure this key is set in Colab secrets
    temperature=0,
    vertexai=True
)

Display jpg images. Alternatively, open the folder icon on the left pannel to see the images.

In [ ]:
from IPython.display import HTML, display
import glob, os

image_paths = glob.glob("*.jpg")
image_paths.sort()
html_content = '<div style="display: flex; flex-wrap: wrap; gap: 20px;">'

for path in image_paths:
    b64 = image_to_base64(path)
    filename = os.path.basename(path) # Clean up path to show just the name

    # Create a vertical column for each image + text
    html_content += f'''
    <div style="display: flex; flex-direction: column; align-items: center;">
        <img src="data:image/jpeg;base64,{b64}" style="height: 300px; border: 1px solid #ddd; margin-bottom: 5px;"/>
        <span style="font-family: monospace; font-size: 14px;">{filename}</span>
    </div>
    '''

html_content += '</div>'

display(HTML(html_content))

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate  # 新增：导入Prompt模板（原有代码可能漏了）
from operator import itemgetter

# ====================== 改动1：修改提示词，同时提取实际金额+折扣前金额 ======================
# 1. 定义针对单张图片的提取 Worker
# 这里的 Prompt 增加了对浮点数精度的强制要求 + 新增折扣前金额提取
single_extract_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个审计级别的 OCR 专家。提取图片中的2个关键金额，以 JSON 格式输出：{{'actual_amount': float, 'original_amount': float}}。规则：1. actual_amount=实际支付的总金额；2. original_amount=无折扣时的原始总金额（无折扣/无该信息则填0）；3. 必须保留小数点后所有数字，严禁截断或四舍五入。"),
    ("human", [
        {"type": "image_url", "image_url": {"url": "{img_url}"}}
    ]),
])

extract_worker = single_extract_prompt | llm | JsonOutputParser()

# 2. 准备你的图片路径列表（5张或更多）
image_paths = [
    "/content/receipt1.jpg",
    "/content/receipt2.jpg",
    "/content/receipt3.jpg",
    "/content/receipt4.jpg",
    "/content/receipt5.jpg"
]

# 3. 使用你现有的逻辑获取 Data URL
# 将所有路径转换为 Base64 数据
image_urls = [get_image_data_url(path) for path in image_paths]

# 4. 构建并行执行链
# 创建一个字典，例如 {'img_0': worker, 'img_1': worker ...}
# 使用 itemgetter(key) 准确地从输入字典中抓取对应的子字典
tasks = {
    f"img_{i}": itemgetter(f"img_{i}") | extract_worker
    for i in range(len(image_urls))
}
parallel_chain = RunnableParallel(**tasks)

# 5. 组装输入数据并执行
# 创建输入字典，例如 {'img_0': {'img_url': 'data:...'}, ...}
chain_input = {f"img_{i}": {"img_url": url} for i, url in enumerate(image_urls)}

# 一键启动并行任务
print(f"正在并行处理 {len(image_paths)} 张图片...")
results = parallel_chain.invoke(chain_input)

# ====================== 改动2：调整汇总逻辑，分别汇总实际金额和折扣前金额 ======================
# 6. 汇总计算（新增折扣前金额汇总）
all_actual_amounts = [res['actual_amount'] for res in results.values()]  # 实际支付金额
all_original_amounts = [res['original_amount'] for res in results.values()]  # 折扣前金额
total_actual = sum(all_actual_amounts)  # 总实际花费
total_original = sum(all_original_amounts)  # 总折扣前金额

print("-" * 30)
print(f"实际金额明细: {all_actual_amounts} | 总实际花费: {total_actual:.2f}")
print(f"折扣前金额明细: {all_original_amounts} | 总折扣前金额: {total_original:.2f}")

# ====================== 新增：查询处理函数（核心作业要求） ======================
def handle_user_query(query):
    """
    处理用户查询，响应2类合法查询，拒绝无关查询
    :param query: 用户输入的文本查询
    :return: 响应文本
    """
    # 统一查询格式：小写 + 去空格，提高匹配率
    clean_query = query.lower().strip()

    # 场景1：查询总花费金额（匹配关键词）
    if any(key in clean_query for key in ["总花费", "总共花了", "total spend", "total paid", "总支付", "实际支付"]):
        return f"✅ 这些账单的总花费金额（实际支付）：{total_actual:.2f}（单位：账单默认货币）"

    # 场景2：查询无折扣的原始金额
    elif any(key in clean_query for key in ["无折扣", "原价", "original amount", "without discount", "折扣前"]):
        if total_original == 0:
            return f"✅ 未识别到折扣信息，无折扣时的金额即为实际总花费：{total_actual:.2f}"
        else:
            return f"✅ 无折扣时需要支付的总金额：{total_original:.2f}（单位：账单默认货币）"

    # 场景3：拒绝无关查询
    else:
        return "❌ 抱歉，我仅支持查询「账单总花费金额」和「无折扣时的原始总金额」，请更换查询内容。"

# ====================== 新增：用户交互循环（作业核心交互要求） ======================
print("\n" + "="*50)
print("🔍 已完成账单金额提取，可开始查询（输入'退出'结束）：")
print("="*50)
while True:
    user_input = input("请输入你的查询：")
    if user_input.lower() == "退出":
        print("查询结束！")
        break
    # 调用查询处理函数并返回结果
    response = handle_user_query(user_input)
    print(f"模型响应：{response}\n")